In [8]:
import torch
import librosa
from importlib import reload
import pandas as pd
import numpy as np
from transformers import Wav2Vec2Processor
import editdistance
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import logging

In [2]:
def calc_cer(hyp, ref):
    score = editdistance.eval(list(hyp), list(ref))
    n_words = len(list(ref))
    return score / n_words

def calc_wer(hyp, ref):
    score = editdistance.eval(hyp.split(), ref.split())
    n_words = len(ref.split())
    return score / n_words

In [9]:
import src.wav2vec
reload(src.wav2vec)
from src.wav2vec import Wav2vec

processor = Wav2Vec2Processor.from_pretrained('assets/processor/')
model = Wav2vec.load_from_checkpoint('weights/cv/last.ckpt', processor=processor)
model.eval()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2vec(
  (model): Wav2Vec2ForCTC(
    (wav2vec2): Wav2Vec2Model(
      (feature_extractor): Wav2Vec2FeatureExtractor(
        (conv_layers): ModuleList(
          (0): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (1): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (2): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (3): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (4): Wav2Vec2LayerNormCo

In [3]:
gt = 'من غلام قمرم غیر قمر هیچ مگو پیش من جز سخن شمع و شکر هیچ مگو سخن رنج مگو جز سخن گنج مگو ور از این بی خبری رنج مبر هیچ مگو دوش دیوانه شدم عشق مرا دید و بگفت آمدم نعره مزن جامه مدر هیچ مگو  ' 
file = 'assets/test.wav'

In [28]:
m_hyp = model.transcribe(file, mode='greedy')
print(m_hyp)

m_cer = calc_cer(m_hyp, gt)
m_wer = calc_wer(m_hyp, gt)
print('cer:', m_cer, ' wer:', m_wer)

من قلام قمرم غیر قمل هیچ مگو پیش من جز سخن شم و شکر هیچ گو سخن رنج مگن شود سخن گنج گو بر از این بی خبری رنج نبر هیچ مگو دو بوش دیوانه شدم عشق مرا دید و ب گفت امدم نه عرم از انجام ب ندار هیچ مگو
cer: 0.15425531914893617  wer: 0.425531914893617


In [27]:
model.setup_beam_decoder(n_beams=100, lm_path='assets/cv6.binary', alpha=0.1, beta=0.1, n_jobs=5)

m_hyp = model.transcribe(file, mode='beam')
print(m_hyp)

m_cer = calc_cer(m_hyp, gt)
m_wer = calc_wer(m_hyp, gt)
print('cer:', m_cer, ' wer:', m_wer)

من قلم عمر غیر عمل هیچ مو پیش من جز سخن شب و شکر هیچ گود سخن رنج نگ شود سخن رنج گ بر از این بی خبر رنج نور هیچ نگ دو پوش دیواره دم عشق مرا دید و به گفته امد ام نه رم از انجام به مدار هیچ مد
cer: 0.26063829787234044  wer: 0.6170212765957447


In [17]:
df = pd.read_csv('/media/data/soroosh/dataset/ASR/cv-fa-6.1/cv-corpus-6.1-2020-12-11/fa/validated.csv')
vocab = set()
for sent in df['sentence']:
    vocab.update(sent.split())
len(vocab)

15830

In [25]:
import src.correct
reload(src.correct)
from src.correct import Corrector

cor = Corrector(list(vocab))

In [26]:
cor.edit(m_hyp)

'من کلام عمرم غیر حمل هیچ لگو پیش من جز سخن شم و شکر هیچ گو سخن رنج مزن شود سخن گنو گو بر از این بی خبری رنج خبر هیچ لگو دو هوش دیوانه شدم عشق م را دید و به گفت امده نه چرم از انجام ب بدار هیچ مداد'

## google speech

In [11]:
import speech_recognition as sr

r = sr.Recognizer()

In [12]:
with sr.AudioFile('assets/test.wav') as source:
    audio = r.record(source)
g_hyp = r.recognize_google(audio, language='fa-IR')
print(g_hyp)

من غلام قمرم غیر قمر هیچ مگو پیش من جز سخن شمع و شکر هیچ مگو سخن رنج مگو جز سخن گنج مگو و از این بی خبری رنج مبر هیچ مگو دیوانه شدم عشق مرا دید و بگفت آمدم نعره مزن جامه مدر هیچ مگو


In [13]:
g_cer = calc_cer(g_hyp, gt)
g_wer = calc_wer(g_hyp, gt)
print('cer:', g_cer, ' wer:', g_wer)

cer: 0.03723404255319149  wer: 0.0425531914893617
